# Testing for upscaling images while keeping the original aspect ratio

In [8]:
from PIL import Image, ImageOps
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import imageio.v3 as iio
import matplotlib.pyplot as plt
import numpy as np
from helper_functions.helper_functions import pixels_between_points
import pathlib
import platform
plt2 = platform.system()
if plt2 == 'Windows': pathlib.PosixPath = pathlib.WindowsPath
import cv2

from scipy.ndimage import map_coordinates

In [38]:
image_path = "D:\Master\MasterProject\model_creation\images\Tiny_not_valid_granule.png"

original_image = Image.open(image_path)
# original_image.thumbnail(size=(1024,1024), resample=Image.Resampling.NEAREST) # Granules are greyscaled, ie heatmaps
cutout_height, cutout_width = np.array(original_image).shape[:2]
print(f"Size ({cutout_height, cutout_width})")
MAX_SIZE = 1024
# Calc max resize while keeping aspect ratio
max_scale_height = int(np.floor(MAX_SIZE / cutout_height))
max_scale_width  = int(np.floor(MAX_SIZE / cutout_width))
scale_factor = min(max_scale_height, max_scale_width)

print(f"Max scaling: ({max_scale_height, max_scale_width})")
print(f"New size ({cutout_width*scale_factor, cutout_height*scale_factor})")
# print(f"New size ({cutout_width*max_scale_width, cutout_height*max_scale_height})")

upscaled_image = original_image.resize((cutout_width*scale_factor, cutout_height*scale_factor), resample=Image.Resampling.NEAREST)
# upscaled_image = original_image.resize((cutout_width*max_scale_width, cutout_height*max_scale_height), resample=Image.Resampling.NEAREST)
original_image

Size ((35, 25))
Max scaling: ((29, 40))
New size ((725, 1015))


In [35]:
# Pad what is left to get (1024, 1024)
image_width, image_height = upscaled_image.size
delta_w = MAX_SIZE - image_width
delta_h = MAX_SIZE - image_height
padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(delta_h//2))
new_im = ImageOps.expand(upscaled_image, padding)
new_im.save("D:\Master\MasterProject\model_creation\images\BIG_not_valid_granule.png")
print(f"Final size: {new_im.size}")

Final size: (1024, 1024)


In [114]:
def scale_point_with_padding(point: np.array, crop_image: np.array, upscale) -> np.array:
    point = np.array(point)
    # --- Scaling values ---
    NEW_MAX_HEIGHT, NEW_MAX_WIDTH = 1024, 1024
    cutout_height, cutout_width = crop_image.shape[:2]
    max_scale_height = int(np.floor(NEW_MAX_HEIGHT / cutout_height))
    max_scale_width  = int(np.floor(NEW_MAX_WIDTH  / cutout_width))
    scale_factor = min(max_scale_height, max_scale_width) # Max amount to scale by while keeping aspect ratio

    # --- Padding values ---
    image_width, image_height = cutout_height*scale_factor, cutout_width*scale_factor # Upscaled image shape 
    delta_w = NEW_MAX_WIDTH  - image_width
    delta_h = NEW_MAX_HEIGHT - image_height

    if upscale:
        # --- Upscale border points ---
        point = point * scale_factor + scale_factor / 2 - 1/2 
        # --- Add padding offset ---
        point[0] += delta_h // 2
        point[1] += delta_w // 2
    else: # Downscale instead
        # --- Remove padding offset ---
        point[0] -= delta_h // 2
        point[1] -= delta_w // 2
        # --- Downscale border points ---
        point = (point - scale_factor / 2 + 1/2) / scale_factor 
    return point

In [124]:
fig = make_subplots(rows=1, cols=2, 
                    horizontal_spacing=0.01, 
                    vertical_spacing=0.1,
                    subplot_titles=('Original image', f"Upscaled image"))
# Add images to plot
base_image_fig = px.imshow(original_image)
fig.add_trace(base_image_fig.data[0], 1, 1)
point1 = (6.5,6.5)
point2 = (2.5,7.5)

fig.add_trace(go.Scatter(x=[point1[0]], y=[point1[1]], marker=dict(color='black', size=10), name=f"Pixel edges", showlegend=False), row=1, col=1)
fig.add_trace(go.Scatter(x=[point2[0]], y=[point2[1]], marker=dict(color='black', size=10), name=f"Pixel edges", showlegend=False), row=1, col=1)

upscaled_image_fig = px.imshow(new_im)
fig.add_trace(upscaled_image_fig.data[0], 1, 2)
point1_up = scale_point_with_padding(point=point1, crop_image=np.array(original_image), upscale=True)
point2_up = scale_point_with_padding(point=point2, crop_image=np.array(original_image), upscale=True)
# ys = scale_point_with_padding(point=ys, crop_image=np.array(original_image), upscale=True)
fig.add_trace(go.Scatter(x=[point1_up[0]], y=[point1_up[1]], marker=dict(color='black', size=10), name=f"Pixel edges", showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=[point2_up[0]], y=[point2_up[1]], marker=dict(color='black', size=10), name=f"Pixel edges", showlegend=False), row=1, col=2)

point2_down = scale_point_with_padding(point=point2_up, crop_image=np.array(original_image), upscale=False)
fig.add_trace(go.Scatter(x=[point2_down[0]], y=[point2_down[1]], marker=dict(color='red', size=10), name=f"Pixel edges", showlegend=False), row=1, col=1)

# fig.add_trace(go.Scatter(x=[2,2], y=[7,7], marker=dict(color='red', size=10), name=f"Point"), row=1, col=1)
# fig.add_trace(go.Scatter(x=[2*scale_factor_x+scale_factor_x/2-0.5,2*scale_factor_x+scale_factor_x/2-0.5], y=[7*scale_factor_y+scale_factor_y/2-0.5,7*scale_factor_y+scale_factor_y/2-0.5], marker=dict(color='red', size=10), name=f"Point"), row=1, col=2)

fig.update_layout(title_text=f"", title_x=0.5, width=1400, height=800, showlegend=True, font_size=11)
fig.update(layout_coloraxis_showscale=False)
fig.show() 

In [94]:
print(351-323)
print(206-178)

print(323+(351-323)/2)
print(178+(206-178)/2)

28
28
337.0
192.0
